In [1]:
#  MERGING RESISTANCE AND SUPPORT IN ONE FUNCTION
def find_trendlines(df,minimum_touch=3, mode="support"):
    final_lines = pd.DataFrame([],columns=["first_point", "last_point", "coefs", "intercepts"])
#     lines_df=pd.DataFrame(columns=["a","b","start","stop","mse","touch", "mse/touch"])
    lines_array=[]
    stockc=df["log_adj_close"].tolist()
    dates=df.index # for converting first_point s to real date .... first_point and last_point s are some indexes of stoko array

    xarr, yarr = find_optimum_points(df["log_adj_close"], ismax = False)

        #finding the best lines 
    # cnt = numbers of line segments
    cnt=0
    # start and stop[cnt] = start poit in x axes and stop point for each line segments
    first_point=[]
    last_point=[]
    checkedpoint=[]
    lastpoint=0
    count_max=0
    a_maxm=[]
    b_maxm=[]
    max_last=0
    # all tangents from 0.04 to -0.04 digree . i will break them in to 1000 pieces this is too short because we're using log numbers
    digree=np.linspace(-0.12,0.12,1000)
    coefs=np.tan(digree*pi / 180)


    for point in range(len(xarr)) :
#         bestline=pd.DataFrame([],columns=["a","b","start","stop","mse","touch", "mse/touch"])
        bestline=[]
        max_last=0  
        count_max=0
    #  x and y are the ones of this point
        x=xarr[point]
        y=yarr[point]
    # for this point i am breaking the line into 100 pieces
#         b_fakes=np.linspace(y-(y*accepted_error),y+(y*accepted_error),100)
        b_fakes=np.linspace(math.log10((10**y)-((10**y)*accepted_error)),math.log10((10**y)+((10**y)*accepted_error)),100)
        for b in b_fakes:
            for a in coefs:
                points_in_touch = []
                count = 0
                lastpoint=0
                for p in range(point+1,len(xarr)):
                #  x_ and y_ s are for all next points in new dimensions !!
                    x_ = xarr[p]-x
                    y_ = yarr[p]
                    if(x_*a+b>=math.log10((10**y_)-((10**y_)*accepted_error))) and (x_*a+b<=math.log10((10**y_)+((10**y_)*accepted_error))):
#                     if((x_*a+b>=y_-(y_*accepted_error)) and (x_*a+b<=y_+(y_*accepted_error))):
                        points_in_touch.append(xarr[p])
                        count+=1
                        lastpoint=xarr[p]
                    elif(mode=="support"):
                        if (math.log10((10**y_)+((10**y_)*accepted_error)) < x_*a+b ): 
                            break
                    elif(mode == "resistance"):
                        if (math.log10((10**y_)+((10**y_)*accepted_error)) > x_*a+b ): 
#                     elif (y_+(y_*accepted_error) < x_*a+b ):
                            break
                
                if(count >= minimum_touch-1):
                    cnt_time[0] += 1
                    
                    mse = mean_squared_error([stockc[x]] +[stockc[indice] for indice in points_in_touch], [b] + [a*(xx-x) + b for xx in points_in_touch])
# THIS IS THE STRUCTURE OF bestline ARRAY AND EACH ROW OF lines_array : [0:COEF, 1:INTERCEPT, 2:first_point_X, 3:STOP_POINT_X, 4:MSE, 5:NUMBER OF TOUCHS, 6:MSE/NUMBER OF TOUCH]

                    lines_array.append([a, a*(-x)+b,x,lastpoint, mse,count + 1, mse / (count + 1)])
                
        if len(lines_array) > 0 :
            linee=np.array(lines_array)
#             CONVERT lines_array TYPE TO NP.ARRAY AIMING TO ITERATE EASILY AND WORK AS A MATRIX 
            min_mt = min(linee[:,6])
    
            for cntt in lines_array:
                if cntt[6] == min_mt:
                    bestline = cntt
                    break
            lines_array = []

    #   if this start poin]t include a line that touch three points: count_max are the number of other points (=2) . so it should be >= minimum touch -1
#       mse of the line should not be more than a specific number 
            if(bestline[5] >= minimum_touch-1) and (bestline[4] < 3e-5 ):
                repetitious = False
                a_in_radian = math.atan(bestline[0])
                a_in_degree = math.degrees(a_in_radian)
                for cnt in range(len(final_lines)):
                    if((math.degrees(math.atan(final_lines.coefs[cnt])) >= a_in_degree - 0.004) and 
                       (math.degrees(math.atan(final_lines.coefs[cnt])) <= a_in_degree + 0.004) and
                       (last_point[cnt]==bestline[3])):
                        repetitious=True

                if(not(repetitious)):
                    final_lines.append({"first_point" : bestline[2], 
                                        "last_point" : bestline[3], 
                                        "coefs" : bestline[0], 
                                        "intercepts" : bestline[1]},
                                       ignore_index = True)

#     dates[startm]  because we wanna return exact date

    return final_lines